In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler

In [2]:
 sc = SparkContext("local[*]", "RFR")
 spark = SparkSession(sc)

In [3]:
training = spark.read.csv("rf_training.csv", inferSchema=True, header=True)
testing = spark.read.csv("rf_testing.csv", inferSchema=True, header=True)
training = training.drop('_c0', 'location', 'date').fillna(0)
dates = testing.select('date').toPandas()['date']
locations = testing.select('location').toPandas()['location']
testing = testing.drop('_c0', 'location', 'date').fillna(0)
feature_cols = training.schema.names[:-1]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
tmp = assembler.transform(training)
training = tmp["features", "new_cases"]

t_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
ttmp = t_assembler.transform(testing)
testing = ttmp["features", "new_cases"]

In [4]:
import time
start = time.time()
tree = RandomForestRegressor(labelCol="new_cases", featuresCol="features", numTrees=1000, impurity='variance')
model = tree.fit(training)
predictions = model.transform(testing)
print(time.time() - start)

24.5563063621521


In [5]:
predictions_col = predictions.select('prediction').toPandas()['prediction'].astype("int")
output = pd.concat([dates, locations, predictions_col], 1)

In [7]:
output.to_csv("spark_predictions")

In [6]:
sc.stop()